In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import lxml

req = requests.get("https://www.espncricinfo.com/records/season/team-match-results/2024-2024?trophy=89")
soup = bs(req.content)

In [4]:
summary_table = soup.find('table')
summary_df = pd.read_html(str(summary_table))[0]

<ipython-input-4-49141edd992e>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(str(summary_table))[0]


# Getting Individual Matches

In [5]:
# Creating an array containing the links of scorecards of all matches
match_links = summary_table.find_all('a',)

href_list = []
for link in match_links:
  href = link.get('href')
  href_list.append(href)

links = href_list[1::2]

# First Match Scorecard:

In [12]:
link = "https://espncricinfo.com" + links[0]
req = requests.get(link)
soup = bs(req.content)



In [27]:
batting_table = soup.find_all('table', class_="ci-scorecard-table")
batting_first = batting_table[0]
batting_second = batting_table[1]

# For batting first:

columns = batting_first.find('thead').find_all('th')
column_names = [c.string for c in columns]

table_rows = batting_first.find('tbody').find_all('tr')
l = []

for tr in table_rows:
  td = tr.find_all('td')
  row = [str(tr.get_text()).strip() for tr in td]
  l.append(row)

df = pd.DataFrame(l, columns=column_names)
df.dropna(inplace=True)
df = df.rename(columns={df.columns[1]: 'Dismissal'})
df['NewColumn'] = 'New Value'
df

,Batting,Dismissal,R,B,M,4s,6s,SR,NewColumn
0,Rohit Sharma (c),c Klaasen b Maharaj,9,5,7,2,0,180.00,New Value
2,Virat Kohli,c Rabada b Jansen,76,59,87,6,2,128.81,New Value
4,Rishabh Pant †,c †de Kock b Maharaj,0,2,2,0,0,0.00,New Value
6,Suryakumar Yadav,c Klaasen b Rabada,3,4,9,0,0,75.00,New Value
8,Axar Patel,run out (†de Kock),47,31,38,1,4,151.61,New Value
10,Shivam Dube,c Miller b Nortje,27,16,32,3,1,168.75,New Value
12,Hardik Pandya,not out,5,2,7,1,0,250.00,New Value
13,Ravindra Jadeja,c Maharaj b Nortje,2,2,1,0,0,100.00,New Value
